# Optimisation of a striatal fast-spiking cell

This notebook demonstartes optimisation of electrophysiological properties of the single-cell model from the mouse striatum [1]. Optimisation procedure follows the algorithm described in [2-4] and uses experimental data from [5].

1. Grillner S, Robertson B (2015) The basal ganglia downstream control of brainstem motor centres — an evolutionarily conserved strategy. Current Opinion in Neurobiology 33:47-52.
2. Van Geit, W., M. Gevaert, G. Chindemi, C. Rössert, J.-D. Courcol, E. Muller, F. Schürmann, I. Segev, and H. Markram (2016). BluePyOpt: Leveraging open source software and cloud infrastructure to optimise model parameters in neuroscience. Front Neuroinform. 10:17.
3. Markram, H., E. Muller, S. Ramaswamy, M. W. Reimann, M. Abdellah, C. A. Sanchez, A. Ailamaki, L. Alonso-Nanclares, N. Antille, S. Arsever, et al. (2015). Reconstruction and simulation of neocortical microcircuitry. Cell 163(2):456–492. 
4. Hines M, Davison A, Muller E (2009) Neuron and Python. Front Neuroinformatics 3:1.
5. Yvonne Johansson and Gilad Silberberg (2016), personal communication.

## Mode of execution

It is recommended to execute all cells of the notebook at once. It takes about 10 minutes to run the default demo optimisation. After optimisation is finished, a user should be able to inspect the text and graphical output of the notebook. Comments above the executable cells explain their function and the output.

## Cell for optimisation

Name of the neuron model for optimisation is predefined and set to the variable `optcell`. It can't be changed at the moment.

In [ ]:
optcell = 'REPLACE_STRIATAL_MODEL_HERE'

## Setting up environment

The neuron model and related files are retreived from the Collab's Storage.

In [ ]:
def get_stored(name, path='', files=[]):
    import os, tarfile
    storage = get_bbp_client().document
    storageBasePath = '/SP6 - Basal ganglia/'
    storagePath = os.path.join(storageBasePath, path)
    archive = name + '.tar.bz2'
    storage.download_file(os.path.join(storagePath, archive), archive)
    with tarfile.open(archive) as tar:
        if files:
            for f in files:
                tar.extract(os.path.join(name, f))
        else:
            tar.extractall('')
    os.remove(archive)

In [ ]:
get_stored(optcell, 'Optimisations')

In [ ]:
import os
os.chdir(optcell)
!ls -F

Neuron mechanisms are compiled for simulation.

In [ ]:
!nrnivmodl mechanisms

Graphical and simulation environment is set up here. BluePyOpt and other Python modules are loaded.

In [ ]:
%load_ext autoreload
%autoreload`

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
import json
import pandas
import numpy as np
import bluepyopt as bpopt
import bluepyopt.ephys as ephys

In [ ]:
from cell_spec import ename, mname, expdata
from utils import *

## Cell model definition

### Morphology

Detailed neuron morphology is used, repaired and corrected for the mouse striatum.

In [ ]:
mpath = os.path.join('morphology', mname) + '.swc'

In [ ]:
import neurom.viewer
neurom.viewer.draw(neurom.load_neuron(mpath));

Morphology file is loaded for optimisation. Reconstructed axon is replaced with a standard 60 &mu;m long initial segment.

In [ ]:
morphology = ephys.morphologies.NrnFileMorphology(mpath,
    do_replace_axon=True)

### Parameters

Model parameter definitions are loaded from the file. File `parameters-demo.json` is used for shorter simulation.

In [ ]:
import cell_model
parameters = cell_model.define_parameters('parameters-demo.json')
for x in parameters: print x

Parameters with bounds as their values will be varied during optimization, other parameters are fixed. 

### Ionic Mechanisms
Load ionic mechanisms from the file `mechanisms.json`.

In [ ]:
mechanisms = cell_model.define_mechanisms('mechanisms.json')

### Neuron
Create the cell model.

In [ ]:
cell = ephys.models.CellModel('fs', 
    morph=morphology, 
    mechs=mechanisms, 
    params=parameters)

Cell parameters which have fixed values are called *frozen*. Collect parameter names of tunable parameters for the optimisation.

In [ ]:
opt_params = [p.name for p in cell.params.values() if not p.frozen]

## Setting up a cell evaluator
### Protocols
Load experimental protocols from the file `protocols.json`.

In [ ]:
import cell_evaluator
protocols = cell_evaluator.define_protocols('protocols.json')

### Features
Load definitions of electrophysiological features from the file `features.json`. Every feature is treated as an objective during optimisation. A specific feature set is defined for each protocol.

In [ ]:
calculator = cell_evaluator.define_fitness_calculator(
    protocols, 'features.json')

### Simulator
Define the simulator to be used for evaluations. Here the Neuron simulator with Python interface by Hines *et al.*  is used.

In [ ]:
simulator = ephys.simulators.NrnSimulator()

### Evaluator
Create the cell evaluator.

In [ ]:
evaluator = ephys.evaluators.CellEvaluator(
    cell_model=cell,
    param_names=opt_params,
    fitness_protocols=protocols,
    fitness_calculator=calculator,
    sim=simulator)

## Optimising the cell
### Optimisation
Specify the number of offspring individuals in each generation and the number of generations for the evolutionary algorithm. Example below uses small numbers for demonstartion, more realistic values which are used on super-computers are commented out.

In [ ]:
offspring_size = 10  # 400
ngenerations = 3     # 100      

Create an optimisation object and run the optimisation. 

In [ ]:
optimiser = bpopt.optimisations.DEAPOptimisation(
    evaluator=evaluator,
    offspring_size=offspring_size)

In [ ]:
pop, hof, log, hist = optimiser.run(max_ngen=ngenerations)

Plot the evolution of the error scores with generations (blue is minimal error, black is average, thin grey line is maximal error, and area between the minimal error and avearage + 1 standard deviation is shaded grey).

In [ ]:
plot_log(log)

Check the parameter values for 10 best models found by the optimizer.

In [ ]:
best_models = []
for record in hof:
    params = evaluator.param_dict(record)
    best_models.append(params)

In [ ]:
pandas.options.display.float_format = '{:,.4g}'.format
pandas.DataFrame(best_models).T

Calculate and visualize relative errors for all features used in optimisation for each of the 10 best models ('individuals').

In [ ]:
evaluate_all = True
best_objectives = []
if evaluate_all:
    for params in best_models:
        objectives = evaluator.evaluate_with_dicts(params)
        best_objectives.append(objectives)
    df = pandas.DataFrame(best_objectives).T
    df = df.sort_index(axis=0, ascending=False)

In [ ]:
if evaluate_all:
    plot_scores(df, figsize=(6, 12))

Plot voltage traces of simulated protocols for a selected individual, 0-9. Individuals are sorted according to their fitness score with individual 0 being the best. Experiment is grey, simulation is blue on the plots.

In [ ]:
individual = 0  # 0..9

In [ ]:
test_parameters = best_models[individual]

In [ ]:
test_responses = evaluator.run_protocols(
    protocols=protocols.values(), 
    param_values=test_parameters)

In [ ]:
plot_responses(test_responses, expdata=expdata, junction_potential=9.5, figsize=(6,12))

Evaluate and plot relative feature errors for the selected individual.

In [ ]:
if evaluate_all:
    objectives = best_objectives[individual]
else:
    objectives = evaluator.evaluate_with_dicts(test_parameters)
plot_objectives(objectives, figsize=(6,12))

User can inspect all 10 best candidate models manually, as was shown above for the best individual 0. To do this, set the variable `individual` to any value from 0 to 9 and execute the cells below the line `individual = ...` as before. Repeat if needed for another individual.